In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [72]:
file_1 = pd.read_csv('data/SeLoger Search Export_20230410_1736.csv')
file_2 = pd.read_csv('data/SeLoger Search Export_20230411_0720.csv')
file_3 = pd.read_csv('data/SeLoger Search Export_20230412_0815.csv')
file_4 = pd.read_csv('data/SeLoger Search Export_20230413_0813.csv')

path = 'data'
csv_files = glob.glob(path + "/*.csv")

df_list = (pd.read_csv(file) for file in csv_files)

data = pd.concat(df_list, axis = 0)

df = data.copy()

df.shape

(3461, 52)

In [73]:
df = df.drop(['id','object', 'annonce_id', 'agency_id', 'agency_page', 'agency_contact_name', 'agency_img_url',
        'agency_phone_number', 'agency_has_email', 'agency_link', 'contact_email', 'description', 'is_redirected',
        'is_expired', 'latitude', 'longitude', 'picture_count', 'photos', 'price_decrease_percent', 'publication_id',
        'pricing_price_note', 'ref', 'title', 'url', 'video_url', 'virtual_visit_url', 'collected_at', 'input_url',
        'input_max_pages', 'input_annonce_details','main_picture','transaction_type', 'short_description', 
             'price_per_meter', 'is_exclusive', 'monthly_price', 'nature', 'estate_type_id', 
              'highlighting_level', 'address', 'bedrooms_count','rooms', 'contact_is_private_seller','business_unit'
              , 'insee_code','position'],
             axis = 1)
df.shape

(3461, 6)

In [74]:
df = df.drop_duplicates().reset_index().drop(['index'], axis = 1)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   area         1580 non-null   float64
 1   district     1194 non-null   object 
 2   estate_type  1585 non-null   object 
 3   postal_code  1585 non-null   int64  
 4   price        1585 non-null   int64  
 5   tags         1585 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 74.4+ KB


In [76]:
#splitting the 'tags' column
#what are all the unique features we can find in it

tag_words = ''
for data in df['tags']:
    tag_words = tag_words + data + ","

unique = []
for word in tag_words.split(","):
    if word not in unique:
        unique.append(word)
        
unique.sort(reverse=True)
print(unique)


['9 pièces', '8 pièces', '7 pièces', '6 pièces', '5 pièces', '4 pièces', '3 pièces', '2 pièces', '11 pièces', '10 pièces', '1 pièce', '0 pièce', ' – m²', ' Étage –/6', ' Étage –/5', ' Étage –/4', ' Étage –/1', ' Étage –/0', ' Étage 9/–', ' Étage 9/9', ' Étage 9/26', ' Étage 9/15', ' Étage 9/13', ' Étage 9/12', ' Étage 9/11', ' Étage 9/10', ' Étage 8/–', ' Étage 8/9', ' Étage 8/8', ' Étage 8/15', ' Étage 7/–', ' Étage 7/9', ' Étage 7/8', ' Étage 7/7', ' Étage 7/4', ' Étage 7/11', ' Étage 7/0', ' Étage 6/–', ' Étage 6/9', ' Étage 6/8', ' Étage 6/7', ' Étage 6/6', ' Étage 6/16', ' Étage 6/13', ' Étage 6/10', ' Étage 6/0', ' Étage 5/–', ' Étage 5/9', ' Étage 5/8', ' Étage 5/7', ' Étage 5/6', ' Étage 5/5', ' Étage 5/14', ' Étage 5/11', ' Étage 5/0', ' Étage 4/–', ' Étage 4/9', ' Étage 4/8', ' Étage 4/7', ' Étage 4/6', ' Étage 4/5', ' Étage 4/4', ' Étage 4/2', ' Étage 4/16', ' Étage 4/14', ' Étage 4/10', ' Étage 4/0', ' Étage 3/–', ' Étage 3/8', ' Étage 3/7', ' Étage 3/6', ' Étage 3/5', ' Ét

In [77]:
#list of the features we can find in 'tags'
#rooms and area are already in the data, so I do not keep them here

features = {'pièce': 'rooms', 'balcon':'balcony', 'ascenseur':'elevator', 
                 'terrasse':'terrace', "étage" : 'floor', 'jardin' : "garden", 
                 'garage' : 'garage','parking' : 'parking_spot', 'box' : 'box' }


def split_column(features, dataframe, column_to_split ):
    
    for key in features :
        list_temp = []

        for data in dataframe[column_to_split] :

                if key.lower() in data.lower() :
                    splited_data = data.split(",")
                    
                    for element in splited_data:
                        
                        if key.lower() in element.lower():
                            list_temp.append(element)
                else :
                    list_temp.append(None)
        dataframe[features[key]] = list_temp


In [78]:
split_column(features, df, 'tags')

In [79]:
df = df.drop(['tags'], axis = 1)

In [80]:
df.tail(3)

,area,district,estate_type,postal_code,price,rooms,balcony,elevator,terrace,floor,garden,garage,parking_spot,box
1582,27.402073,NaN,Studio,69005,156000,1 pièce,None,None,None,Étage 1/4,None,None,None,None
1583,51.999107,Montchat,Appartement,69003,232800,2 pièces,Balcon,Ascenseur,None,Étage 2/4,None,None,None,Box
1584,118.002846,Plateau de la Croix Rousse-Saône,Appartement,69004,995000,5 pièces,Balcon,Ascenseur,Terrasse,Étage 7/8,None,None,Parking,None


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          1580 non-null   float64
 1   district      1194 non-null   object 
 2   estate_type   1585 non-null   object 
 3   postal_code   1585 non-null   int64  
 4   price         1585 non-null   int64  
 5   rooms         1585 non-null   object 
 6   balcony       439 non-null    object 
 7   elevator      968 non-null    object 
 8   terrace       242 non-null    object 
 9   floor         1449 non-null   object 
 10  garden        60 non-null     object 
 11  garage        108 non-null    object 
 12  parking_spot  230 non-null    object 
 13  box           231 non-null    object 
dtypes: float64(1), int64(2), object(11)
memory usage: 173.5+ KB


In [82]:
#creating 3 categories of floor
#Ground floor, Last floor, or Neither

list_floor = []

for floor in df['floor'] :
    
    if floor == None :
        list_floor.append('Other')
    elif (str(floor.replace('Étage ',''))[1] == '0'):
        list_floor.append('Ground Floor')
    elif (str(floor.replace('Étage ',''))[1] == str(floor.replace('Étage ',''))[3]) :
        list_floor.append('Last Floor')
    else :
        list_floor.append('Other')

        
df['floor'] = list_floor 

df['floor'].value_counts(normalize=True)

Other           0.786120
Last Floor      0.123028
Ground Floor    0.090852
Name: floor, dtype: float64

In [83]:
df['area'] = df['area'].values.round()

In [84]:
#I want to use this column to know if the flat is new or not

list_new = []

for estate_type in df['estate_type']:
    if 'neuf' in estate_type.lower() :
        list_new.append(True)
    else : 
        list_new.append(False)
        
df['new_flat'] = list_new

In [85]:
df = df.drop(['estate_type'], axis = 1)

In [86]:
df['new_flat'].value_counts(normalize=True)

False    0.887066
True     0.112934
Name: new_flat, dtype: float64

In [87]:
#list of all features I will keep on True/False option

list_t_f_features = ['balcony', 'elevator', 'garden', 'terrace', 'garage', 'box', 'parking_spot']

def true_false(dataframe, column) :
    dataframe[column] = dataframe[column].notnull().astype(bool)
    dataframe[column] = dataframe[column].fillna(False)

In [88]:
for features in list_t_f_features :
    true_false(df, features)

In [89]:
#merging box and garage
df['garage'] = df['garage'] | df['box'] 

df = df.drop(['box'], axis = 1)

In [90]:
df

,area,district,postal_code,price,rooms,balcony,elevator,terrace,floor,garden,garage,parking_spot,new_flat
0,19.0,Croix Paquet,69001,175000,1 pièce,False,False,False,Other,False,False,False,False
1,19.0,La Guillotière Sud,69007,133000,1 pièce,False,False,False,Other,False,True,False,False
2,39.0,NaN,69006,237000,2 pièces,True,True,True,Other,False,False,False,False
3,99.0,Montchat,69003,755000,5 pièces,True,True,False,Other,False,True,False,False
4,48.0,Montchat,69003,220000,2 pièces,True,False,True,Ground Floor,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,65.0,Gerland,69007,389000,3 pièces,False,False,False,Other,False,False,False,True
1581,85.0,Fourvière-Saint Just,69005,349000,4 pièces,True,True,True,Other,False,True,False,False
1582,27.0,NaN,69005,156000,1 pièce,False,False,False,Other,False,False,False,False
1583,52.0,Montchat,69003,232800,2 pièces,True,True,False,Other,False,True,False,False


In [91]:
#area and district have missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          1580 non-null   float64
 1   district      1194 non-null   object 
 2   postal_code   1585 non-null   int64  
 3   price         1585 non-null   int64  
 4   rooms         1585 non-null   object 
 5   balcony       1585 non-null   bool   
 6   elevator      1585 non-null   bool   
 7   terrace       1585 non-null   bool   
 8   floor         1585 non-null   object 
 9   garden        1585 non-null   bool   
 10  garage        1585 non-null   bool   
 11  parking_spot  1585 non-null   bool   
 12  new_flat      1585 non-null   bool   
dtypes: bool(7), float64(1), int64(2), object(3)
memory usage: 85.3+ KB


In [92]:
#area has very few missing data, I can drop those few rows
df = df.dropna(subset=['area'])

In [93]:
df = df.loc[df['rooms']!='0 pièce']

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1579 entries, 0 to 1584
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          1579 non-null   float64
 1   district      1188 non-null   object 
 2   postal_code   1579 non-null   int64  
 3   price         1579 non-null   int64  
 4   rooms         1579 non-null   object 
 5   balcony       1579 non-null   bool   
 6   elevator      1579 non-null   bool   
 7   terrace       1579 non-null   bool   
 8   floor         1579 non-null   object 
 9   garden        1579 non-null   bool   
 10  garage        1579 non-null   bool   
 11  parking_spot  1579 non-null   bool   
 12  new_flat      1579 non-null   bool   
dtypes: bool(7), float64(1), int64(2), object(3)
memory usage: 97.1+ KB


In [95]:
sorted_df = df.sort_values('postal_code')

In [96]:
sorted_df['district'].fillna(method='bfill', inplace = True)

In [97]:
sorted_df['district'].isna().sum().any()

False

In [98]:
df = sorted_df.sort_values('price').reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579 entries, 0 to 1578
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          1579 non-null   float64
 1   district      1579 non-null   object 
 2   postal_code   1579 non-null   int64  
 3   price         1579 non-null   int64  
 4   rooms         1579 non-null   object 
 5   balcony       1579 non-null   bool   
 6   elevator      1579 non-null   bool   
 7   terrace       1579 non-null   bool   
 8   floor         1579 non-null   object 
 9   garden        1579 non-null   bool   
 10  garage        1579 non-null   bool   
 11  parking_spot  1579 non-null   bool   
 12  new_flat      1579 non-null   bool   
dtypes: bool(7), float64(1), int64(2), object(3)
memory usage: 84.9+ KB


## End of Global Data Cleaning / Analysis

 ## Preprocessing

In [99]:
from sklearn.preprocessing import LabelEncoder

In [100]:
encoder = LabelEncoder()

In [101]:
for data in df[['balcony', 'elevator', 'terrace', 'garden', 'garage', 'parking_spot', 'new_flat']]:
    df[data] = encoder.fit_transform(df[data])

for data in df[['postal_code', 'district', 'rooms', 'floor']]:
    df = df.join(pd.get_dummies(df[data])).drop([data], axis = 1)
    


In [106]:
pd.set_option('display.max_column',52)
df[df['9 pièces'] == True]

,area,price,balcony,elevator,terrace,garden,garage,parking_spot,new_flat,69001,69002,69003,69004,69005,69006,69007,69008,69009,Bellecour-Ainay,Champvert-Point du Jour-Saint Irénée,Cordeliers-Célestins-Bourse,Croix Paquet,Croix Rousse-Centre,Croix Rousse-Gros Caillou,Fourvière-Saint Just,Gerland,...,Plateau de la Croix Rousse-Saône,Point du Jour-Ménival,Saint Rambert,Saint Vincent,Sans Souci-Dauphiné,Terreaux,Transvaal,Tête d'Or-Foch-Vitton,Vaise-Rochecardon-Industrie,Vieux Lyon,Villette,États-Unis,1 pièce,10 pièces,11 pièces,2 pièces,3 pièces,4 pièces,5 pièces,6 pièces,7 pièces,8 pièces,9 pièces,Ground Floor,Last Floor,Other
1560,231.0,1490000,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1577,384.0,2950000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [107]:
#since what is inside the flat seems more important than the location, I will only keep the district
df.drop([69001,69002,69003,69004,69005,69006,69007,69008,69009], axis = 1, inplace= True)

In [108]:
pd.set_option('display.max_column',62)
df.head()

,area,price,balcony,elevator,terrace,garden,garage,parking_spot,new_flat,Bellecour-Ainay,Champvert-Point du Jour-Saint Irénée,Cordeliers-Célestins-Bourse,Croix Paquet,Croix Rousse-Centre,Croix Rousse-Gros Caillou,Fourvière-Saint Just,Gerland,Gorge du Loup-Champvert,Jean Macé,La Duchère,La Guillotière Nord,La Guillotière Sud,La Plaine,Laënnec-Mermoz,Le Grand Trou,Les Brotteaux-Bellecombe-Masséna,Monplaisir-Le Bachut,Montchat,Part Dieu,Perrache-Le Confluent,Plateau de la Croix Rousse-Ouest,Plateau de la Croix Rousse-Saône,Point du Jour-Ménival,Saint Rambert,Saint Vincent,Sans Souci-Dauphiné,Terreaux,Transvaal,Tête d'Or-Foch-Vitton,Vaise-Rochecardon-Industrie,Vieux Lyon,Villette,États-Unis,1 pièce,10 pièces,11 pièces,2 pièces,3 pièces,4 pièces,5 pièces,6 pièces,7 pièces,8 pièces,9 pièces,Ground Floor,Last Floor,Other
0,11.0,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,8.0,52000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,25.0,60000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,10.0,66000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,18.0,69000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [109]:
#normalize the price column
#df['price'] = np.log(df['price'] + 1)

In [110]:
#df['price'].hist(figsize=(10,5))

## Machine Learning Model

In [111]:
from sklearn.model_selection import train_test_split

In [119]:
seed = 42

In [120]:
X = df.drop(['price'], axis = 1)
y = df['price']

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, train_size=0.8, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=seed)

In [133]:
from sklearn.ensemble import RandomForestRegressor

In [134]:
r_forest = RandomForestRegressor()

r_forest.fit(X_train, y_train)
r_forest.score(X_val, y_val)

0.8396597095069718

In [135]:
r_forest.score(X_test, y_test)

0.7839579677334578

In [136]:
df.head(2)

,area,price,balcony,elevator,terrace,garden,garage,parking_spot,new_flat,Bellecour-Ainay,Champvert-Point du Jour-Saint Irénée,Cordeliers-Célestins-Bourse,Croix Paquet,Croix Rousse-Centre,Croix Rousse-Gros Caillou,Fourvière-Saint Just,Gerland,Gorge du Loup-Champvert,Jean Macé,La Duchère,La Guillotière Nord,La Guillotière Sud,La Plaine,Laënnec-Mermoz,Le Grand Trou,Les Brotteaux-Bellecombe-Masséna,Monplaisir-Le Bachut,Montchat,Part Dieu,Perrache-Le Confluent,Plateau de la Croix Rousse-Ouest,Plateau de la Croix Rousse-Saône,Point du Jour-Ménival,Saint Rambert,Saint Vincent,Sans Souci-Dauphiné,Terreaux,Transvaal,Tête d'Or-Foch-Vitton,Vaise-Rochecardon-Industrie,Vieux Lyon,Villette,États-Unis,1 pièce,10 pièces,11 pièces,2 pièces,3 pièces,4 pièces,5 pièces,6 pièces,7 pièces,8 pièces,9 pièces,Ground Floor,Last Floor,Other
0,11.0,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,8.0,52000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [137]:
import pickle

In [138]:
pickle.dump(r_forest, open('model.pkl', 'wb'))